In [1]:
import numpy as np

from sklearn.metrics import mean_squared_error

In [2]:
result_dir = '../results/'

# Table functions

In [3]:
table = ''
block_entries = []

In [11]:
indent = '   '
col_sep = ' & '
end_row = r'\\'
midrule = r'\midrule'
end_line = '\n'
header = (r'Algorithm & Setup'
          + r' & $\MSE_\boldx \left[ \log r(\boldx; \boldtheta_1, \boldtheta_0) \right]$'
          + r' & $\MSE_\boldtheta \left[ E[\log r(\boldx; \boldtheta, \boldtheta_0)] \right]$'
          + r' & $\MSE_\boldtheta \left[ \Delta E[\log r(\boldx; \boldtheta, \boldtheta_0)] \right]$')
emphasis_begin = r'\mathbf{'
emphasis_end = r'}'

def init():
    global table, block_entries
    table = indent + header + end_row + end_line + indent + midrule + end_line
    block_entries = []
    
    
def finalise_block():
    global table, block_entries
    
    n_metrics = len(block_entries[0]) - 2
    
    # Find best performance
    block_metrics = [line[2:] for line in block_entries]
    block_metrics = np.array(block_metrics)
    block_best = [np.nanargmin(block_metrics[:,i]) for i in range(n_metrics)]
    
    # Format entries
    text = ''
    
    for i, line in enumerate(block_entries):
        text += indent + line[0] + col_sep + line[1] + col_sep
        for j in range(n_metrics):
            if np.isfinite(line[j + 2]):
                text += format_number(line[j + 2], 2, emphasize=(i == block_best[j]))
            if j == len(line) - 3:
                text += end_row + end_line
            else:
                text += col_sep
                
    # Add to document and reset for next block
    table += text
    block_entries = []
    
    
def new_block():
    global table, block_entries
    
    finalise_block()
    table += indent + midrule + end_line
    
    
def format_number(number,
                  precision=2,
                  trailing_zeros=True,
                  fix_minus_zero=True,
                  latex_math_mode=True,
                  emphasize=False):
    if precision == 0:
        temp =  str(int(round(number,precision)))
    elif trailing_zeros:
        temp =  ('{:.' + str(precision) + 'f}').format(round(number,precision))
    else:
        temp =  str(round(number,precision))
    if fix_minus_zero and len(temp) > 0:
        if temp[0] == '-' and float(temp) == 0.:
            temp = temp[1:]
    if latex_math_mode:
        if emphasize:
            temp = '$\mathbf{' + temp + '}$'
        else:
            temp = '$' + temp + '$'
    elif emphasize:
        temp = r'\emph{' + temp + r'}'
    return temp
    
    
def add(col1, col2, filename):
    global table, block_entries
          
    line = [col1, col2]
    
    # Calculate mean squared error for fixed theta pair
    try:
        log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
        log_r_estimated = np.log(np.load(result_dir + 'parameterized/r_nottrained_' + filename + '.npy'))
        line.append(mean_squared_error(log_r_truth, log_r_estimated))
    except (IOError, ValueError):
        line.append(np.nan)
    
    # Calculate mean squared error over all theta
    try:
        expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
        expected_log_r_estimated = np.load(result_dir + 'parameterized/llr_' + filename + '.npy')
        line.append(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except (IOError, ValueError):
        line.append(np.nan)
    
    # Calculate mean squared error over all theta after subtracting minimum
    try:
        expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
        expected_log_r_truth -= np.min(expected_log_r_truth)
        expected_log_r_estimated = np.load(result_dir + 'parameterized/llr_' + filename + '.npy')
        expected_log_r_estimated -= np.min(expected_log_r_estimated)
        line.append(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except (IOError, ValueError):
        line.append(np.nan)
        
    print(line)
    block_entries.append(line)
    
    
def finalise():
    finalise_block()
    print(table)

# Make table

In [12]:
labels = ['carl (raw)', 'carl (calibrated)',
          'score (raw)', 'score (calibrated)',
          'carl + score (raw)', 'carl + score (calibrated)',
          'regression (raw)', 'regression + score (raw)']

filenames = ['carl', 'carl_calibrated',
             'score', 'score_calibrated',
             'combined', 'combined_calibrated',
             'regression', 'combinedregression']

In [13]:
init()

for i, (l, f) in enumerate(zip(labels[:4], filenames[:4])):
    
    if i > 0:
        new_block()
        
    add(l, 'Baseline (2)', f + '_shallow')
    add('', 'Baseline (3)', f)
    add('', 'Baseline (5)', f + '_deep')
    
    add('', r'Random $\boldtheta$ (2)', f + '_random_shallow')
    add('', r'Random $\boldtheta$ (3)', f + '_random')
    add('', r'Random $\boldtheta$ (5)', f + '_random_deep')
    
    add('', 'Physics-aware (1)', f + '_aware_shallow')
    add('', 'Physics-aware (2)', f + '_aware')
    add('', 'Physics-aware (3)', f + '_aware_deep')

finalise()

['carl (raw)', 'Baseline (2)', 0.10527852108832415, 1.2652737797865679, 1.1641490069206144]
['', 'Baseline (3)', 0.17103436934952676, 11.431393447319008, 1.6033376397865375]
['', 'Baseline (5)', 0.2354620403777036, 8.79697507695399, 1.7302440775814498]
['', 'Random $\\boldtheta$ (2)', 0.1344605545998348, 3.121277160802298, 0.39171464062855565]
['', 'Random $\\boldtheta$ (3)', 0.1667068167763629, 4.958233092264789, 2.204971724969566]
['', 'Random $\\boldtheta$ (5)', 0.288225637244141, 32.019672375612714, 1.2293805101888304]
['', 'Physics-aware (1)', nan, nan, nan]
['', 'Physics-aware (2)', nan, nan, nan]
['', 'Physics-aware (3)', nan, nan, nan]
['carl (calibrated)', 'Baseline (2)', 0.10020529103783438, 0.28864958262324036, 0.30675035132118605]
['', 'Baseline (3)', 0.1374366064829354, 0.43604213891103794, 0.5393850319581848]
['', 'Baseline (5)', 0.2293383443375764, 0.7084327818126889, 1.1374787122050793]
['', 'Random $\\boldtheta$ (2)', 0.10933914438388546, 0.24432037208961074, 0.2608362

/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/ipykernel/__main__.py:87: RuntimeWarning: invalid value encountered in log
/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)


In [14]:
init()

for i, (l, f) in enumerate(zip(labels[4:], filenames[4:])):
    
    if i > 0:
        new_block()
        
    add(l, 'Baseline (2)', f + '_shallow')
    add('', 'Baseline (3)', f)
    add('', 'Baseline (5)', f + '_deep')
    
    add('', r'Random $\boldtheta$ (2)', f + '_random_shallow')
    add('', r'Random $\boldtheta$ (3)', f + '_random')
    add('', r'Random $\boldtheta$ (5)', f + '_random_deep')
    
    add('', 'Physics-aware (1)', f + '_aware_shallow')
    add('', 'Physics-aware (2)', f + '_aware')
    add('', 'Physics-aware (3)', f + '_aware_deep')

finalise()

['carl + score (raw)', 'Baseline (2)', 0.032382830791463056, 1.623569591224714, 0.22625764158349254]
['', 'Baseline (3)', 0.05887562571563303, 6.662388658323162, 0.6274901203404865]
['', 'Baseline (5)', 0.38435591131421715, 92.42710439508038, 0.9188507919471646]
['', 'Random $\\boldtheta$ (2)', 0.19218767436825118, 2.5115309131550374, 2.9926185436949435]
['', 'Random $\\boldtheta$ (3)', 0.1961802097405059, 2.287188542319481, 3.891902876814821]
['', 'Random $\\boldtheta$ (5)', 0.23448951660292652, 6.874308466751822, 2.7639115552369553]
['', 'Physics-aware (1)', nan, nan, nan]
['', 'Physics-aware (2)', nan, nan, nan]
['', 'Physics-aware (3)', nan, nan, nan]
['carl + score (calibrated)', 'Baseline (2)', 0.040272716792260355, 0.2179823869541931, 0.21276728574890325]
['', 'Baseline (3)', 0.049169981121813956, 0.29770274949008, 0.3119821573362023]
['', 'Baseline (5)', 0.06798132709298858, 0.3664275715134106, 0.28071153957813305]
['', 'Random $\\boldtheta$ (2)', 0.12705998459496692, 0.8589920

/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/ipykernel/__main__.py:87: RuntimeWarning: invalid value encountered in log
/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
